In [1]:
import mediapipe as mp
import cv2
import numpy as np

# draw landmarks & connections to screen
mp_drawing = mp.solutions.drawing_utils
# import Pose model
mp_pose = mp.solutions.pose

In [2]:
def calc_angle(x, y, z):
    x = np.array(x)
    y = np.array(y)
    z = np.array(z)

    radians = np.arctan2(z[1]-y[1], z[0]-y[0]) - np.arctan2(x[1]-y[1], x[0]-y[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [3]:
# def rescale_frame(frame, height, width, percent=75):
#     width = int(width * percent / 100)
#     height = int(height * percent / 100)
#     dim = (width, height)
#     return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)

In [8]:
def recognise_squat(detection):
    
    global counter
    global state
    global feedback
    global range_flag
       
    try:
        landmarks = detection.pose_landmarks.landmark
        
        # GET COORDINATES
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
        
        right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        
        # CALCULATE ANGLE 
        angle = calc_angle(left_hip, left_knee, left_heel)
        
        #POSE CHECKING 1: Knees bending inwards    
        shoulder_dist = left_shoulder[0] - right_shoulder[0]
        knee_dist = left_knee[0] - right_knee[0]
        
        print(shoulder_dist - knee_dist)
        
        if abs(shoulder_dist - knee_dist) > 0.07:
            feedback = 'Open up your knees further apart (shoulder width)!'
        else:
            feedback = ''
            
        # standing up
        if angle > 170:
            state = 'Up'
            
        if angle < 120 and state == "up":
            state = "down"
            # POSE CHECKING 2: Not squatting low enough (knees to hips level)
            if left_hip[1] - left_knee[1] < -0.1:
                feedback = 'Did not squat low enough.'
            else:
                counter += 1
        
    
        ## ================================================================
        
        # down state
#         if left_elbow_angle > 160 and right_elbow_angle > 160:
#             if not range_flag:
#                 feedback = 'Did not curl completely.'
#             else:
#                 feedback = 'Good rep!'
#             state = 'Down'
#         
        # not fully curled
#         elif (left_elbow_angle > 50 and right_elbow_angle > 50) and state == 'Down':
#             range_flag = False
#             feedback = ''
#         
        # up state
#         elif (left_elbow_angle < 30 and right_elbow_angle < 30) and state == 'Down':
#             state = 'Up'
#             feedback = ''
#             range_flag = True
#             counter += 1
    
    except:
        pass

In [9]:
# initialise variables
counter = 0
state = 'Down'
range_flag = True
feedback = ''
frame_count = 0

feed = cv2.VideoCapture(0)

# Get user's maximum resolution
WIDTH = 10000
HEIGHT = 10000
feed.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
feed.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)
width = int(feed.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(feed.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Mediapipe Pose model instance
with mp_pose.Pose(min_detection_confidence=50, min_tracking_confidence=50) as pose:
    while feed.isOpened():
        ret, frame = feed.read()
        frame_count += 1
        # Mirror frame
        frame = cv2.flip(frame, 1)
#         frame = rescale_frame(frame, height, width, percent = 100)
        # Recolor image from BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Pose detection
        detection = pose.process(image)
        # Recolor image from RGB back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = True

        # Render detections
        mp_drawing.draw_landmarks(image, detection.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        # Recognise particular exercise [REPLACABLE]
        recognise_squat(detection)

        # Status box setup
        cv2.rectangle(image, (0,0), (width, int(height*0.1)), (245,117,16), -1)
        cv2.putText(image, "REPS:", (int(width*0.01), int(height*0.025)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA) # font, size, color, line width, line type
        
        cv2.putText(image, "STATE:", (int(width*0.1), int(height*0.025)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, "FEEDBACK:", (int(width*0.2), int(height*0.025)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, str(counter), (int(width*0.01), int(height*0.08)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.putText(image, state, (int(width*0.1), int(height*0.08)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.putText(image, feedback, (int(width*0.2), int(height*0.08)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        window_name = 'Exercise Counter'
        
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        cv2.imshow(window_name, image)
        
        # quit webcam
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()
cv2.waitKey(1)
feed.release()

0.008630722761154175
0.008654773235321045
0.013899296522140503
0.022487938404083252
0.03039693832397461
0.0355876088142395
0.03656211495399475
0.03604543209075928
0.03832593560218811
0.0383775532245636
0.033646196126937866
0.034136414527893066
0.03524795174598694
0.03642773628234863
0.03829681873321533
0.036768943071365356
0.034845829010009766
0.034203290939331055
0.031594425439834595
0.023218244314193726
0.017109930515289307
0.0026803314685821533
-0.0088881254196167
-0.014397680759429932
-0.013302803039550781
-0.01153099536895752
-0.005069732666015625
-0.002442598342895508
-0.0022228658199310303
-0.0022344589233398438
0.00011470913887023926
0.0012868046760559082
0.002698570489883423
0.0054931640625
0.0075501203536987305
0.009081721305847168
0.009164392948150635
0.008004963397979736
0.0071201324462890625
0.0070743560791015625
0.006670355796813965
0.005619555711746216
0.004109472036361694
0.0002970695495605469
-0.00069427490234375
-0.0020450949668884277
-0.0032028555870056152
-0.0013155

0.007586061954498291
0.007219076156616211
0.006907522678375244
0.007034868001937866
0.006930828094482422
0.007158517837524414
0.0071561336517333984
0.0072412192821502686
0.007127940654754639
0.007193148136138916
0.007105052471160889
0.007193475961685181
0.007120102643966675
0.00705370306968689
0.006901800632476807
0.00694429874420166
0.006967514753341675
0.00705990195274353
0.007372438907623291
0.00751042366027832
0.007601141929626465
0.0076600611209869385
0.007999986410140991
0.008125126361846924
0.008302479982376099
0.00834617018699646
0.008212119340896606
0.008234739303588867
0.008197486400604248
0.008202791213989258
0.008539855480194092
0.008643686771392822
0.008773475885391235
0.008795112371444702
0.008750051259994507
0.008738517761230469
0.008784204721450806
0.00899997353553772
0.00859057903289795
0.008728235960006714
0.008638828992843628
0.008536964654922485
0.008487522602081299
0.008481204509735107
0.008322536945343018
0.008664757013320923
0.00861862301826477
0.0082768201828002

0.051950305700302124
0.04999136924743652
0.04507887363433838
0.04919856786727905
0.0474531352519989
0.04667770862579346
0.04554271697998047
0.04176715016365051
0.042030274868011475
0.046461284160614014
0.04672083258628845
0.05013516545295715
0.0513380765914917
0.051570355892181396
0.05163264274597168
0.05441880226135254
0.04826703667640686
0.05212298035621643
0.056880056858062744
0.05881786346435547
0.058437198400497437
0.021695107221603394
0.04993531107902527
0.06079012155532837
0.06447023153305054
0.0656556487083435
0.07117918133735657
0.07420432567596436
0.07262641191482544
0.07042154669761658
0.06637248396873474
0.06612420082092285
0.06668823957443237
0.06974628567695618
0.06876617670059204
0.06733673810958862
0.0689079761505127
0.0692892074584961
0.07004961371421814
0.06838342547416687
0.06848779320716858
0.06878873705863953
0.0680127739906311
0.06648778915405273
0.06605535745620728
0.06606438755989075
0.06515061855316162
0.06428682804107666
0.06472077965736389
0.0646185576915741


0.05750066041946411
0.05470740795135498
0.055232077836990356
0.056219905614852905
0.05410781502723694
0.054776906967163086
0.054138004779815674
0.054107666015625
0.05324065685272217
0.052697211503982544
0.05146345496177673
0.05056142807006836
0.05018824338912964
0.05181971192359924
0.055263519287109375
0.0563206672668457
0.05577796697616577
0.053899019956588745
0.05465799570083618
0.05599376559257507
0.05424511432647705
0.05273157358169556
0.05253523588180542
0.0533122718334198
0.05270564556121826
0.05197620391845703
0.054109036922454834
0.056237637996673584
0.05734443664550781
0.05733773112297058
0.05619439482688904
0.052316635847091675
0.04764261841773987
0.05219122767448425
0.05639469623565674
0.060777485370635986
0.06261590123176575
0.07506603002548218
0.09603291749954224
0.06696230173110962
0.10203135013580322
0.11257165670394897
0.10786408185958862
0.1111534833908081
0.10812777280807495
0.11494570970535278
0.12107086181640625
0.12721943855285645
0.12407857179641724
0.117219209671